### THE STAR METHOD
The STAR method is a structured manner of responding to a behavioral-based interview question by
discussing the specific situation, task, action, and result of the situation you are describing.

* Situation: Describe the situation that you were in or the task that you needed to accomplish. You
must describe a specific event or situation, not a generalized description of what you have done in
the past. Be sure to give enough detail for the interviewer to understand. This situation can be
from a previous job, from a volunteer experience, or any relevant event.
* Task: What goal were you working toward?
* Action: Describe the actions you took to address the situation with an appropriate amount of
detail and keep the focus on YOU. What specific steps did you take and what was your particular
contribution? Be careful that you don’t describe what the team or group did when talking about a
project, but what you actually did. Use the word “I,” not “we” when describing actions.
* Result: Describe the outcome of your actions and don’t be shy about taking credit for your
behavior. What happened? How did the event end? What did you accomplish? What did you
learn? Make sure your answer contains multiple positive results.

Make sure that you follow all parts of the STAR method. Be as specific as possible at all times, without
rambling or including too much information.

In [1]:
!pip install -U -qq transformers accelerate sentence-transformers

In [2]:
from huggingface_hub import notebook_login, Repository

# Login to Hugging Face
notebook_login()

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import time

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto") 
                                             
tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                          use_fast=True) 

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [5]:
situation = """
Create an impact group where we remove loneliness and improve mental health
for people who are alone such as remote workers, single parents and parents whose kids have left their home.

"""

task = """
My aim is to create events that are fun, socialable, explore Dublin and experience different cultures. 
The events where people will hesitate to go alone.


"""

action = """
Focus on different cultures e.g., Italian, Spanish Irish etc. 
Create different kind of events including slow friending, games, picnics, generative AI for audio guided museum tours.
Meet regularly ensuring there is an event every weekend.
Free of cost

"""

result = """
In 1 year I have organised over 150 events.
There are over 2000 members in the group. 30% of them are active members.
4.7 star rating out of over 600 reviews.


"""

In [6]:
resp_messages = [
{"role": "system", 
"content": """

Your task is to write the text delimited in triple quotes as a short STAR Interview Technique. 
The STAR interview technique is part of a strategy called behavioural-based interviewing. Behavioural interviews assess your past experience by asking you to provide specific examples of your skills, abilities, and competencies. 

Situation — Explain what the issue is. 
Open with a brief description of the Situation and context of the success story (who, what, where, when, how).
What is the problem?
What needs to be accomplished?
What restrictions are present?
What business outcomes are expected?

Task — Describe the responsibilities needed. 
Explain the Task you had to complete highlighting any specific challenges or constraint
What needs to be done?
Who needs to be doing these things?
How soon do they need to be done?
What goal were you working toward?

Action — Describe the actions you took to address the situation with an appropriate amount of
detail.
Keep the focus on YOU. 
What specific steps did you take and what was your particular
contribution? 
Be careful that you don’t describe what the team or group did when talking about a
project, but what you actually did. 
Use the word “I,” not “we” when describing actions. 

Result - Describe the outcome of your actions and don’t be shy about taking credit for your
behavior. 
What happened? 
How did the event end? 
What did you accomplish? What did you learn? 
Make sure your answer contains multiple positive results. 

Make sure that you follow all parts of the STAR method. 
Be as specific as possible at all times, without rambling or including too much information. 
Eliminate any examples that do not paint you in a
positive light. 
However, keep in mind that some examples that have a negative result can highlight your strengths in the face of adversity. 

You also check for spelling and grammar mistakes and make sentences more fluent.
You rewrite in an active voice. 
Make sure that the tone is professional, concise yet informal.
Text should flow properly for a human recruiter to grasp the information as quickly as possible. 
Keep the terms simple and without jargons. 
Limit the text in maximum 2000 words.


"""},
{"role": "user", 
"content": f"""

## Situation 
{situation} 

## Task
{task}

## Actions
{action}

## Result
{result}

Could you please write me an answer to a behavioral interview question using STAR technique?


"""},
]
start = time.time()

resp_input_ids = tokenizer.apply_chat_template(
    resp_messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda:0")

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

resp_outputs = model.generate(
    resp_input_ids,
    max_new_tokens=4096,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
resp_response = resp_outputs[0][resp_input_ids.shape[-1]:]
print(tokenizer.decode(resp_response, skip_special_tokens=True))
end = time.time()
print(f"Time (minutes): {(end - start)}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
2024-05-15 13:03:28.539666: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-15 13:03:28.539799: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-15 13:03:28.657231: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Here's an answer to a behavioral interview question using the STAR technique:

Interviewer: "Can you tell me about a time when you created a group or initiative that had a positive impact on people's lives?"

Here's my answer:

**Situation**: As I was working remotely, I noticed that many people around me were struggling with loneliness and mental health issues. I realized that this was a common problem among remote workers, single parents, and parents whose kids had left their home. I wanted to create a solution that would bring people together and provide a sense of community.

**Task**: My aim was to create a group that would host fun, social, and cultural events in Dublin, allowing people to explore the city and experience different cultures. I wanted to create events that people would hesitate to attend alone, but would feel comfortable attending with others.

**Action**: I focused on creating a diverse range of events that would cater to different interests and cultures. I organi